# Установка

In [ ]:
# @title Установка пакетов
from IPython.display import clear_output

!pip install --upgrade tiktoken
!pip install langchain openai
!pip install faiss-cpu

clear_output()

import getpass
import openai
import os
def get_key_ОpenAI():
  openai.api_key = getpass.getpass(prompt='Введите секретный ключ для сервиса chatGPT: ')
  os.environ["OPENAI_API_KEY"] = openai.api_key

get_key_ОpenAI()

Введите секретный ключ для сервиса chatGPT: ··········


In [ ]:
#@title Импорт библиотек и Сервисные функции
from IPython.display import clear_output
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

import tiktoken
import re


# ----------------------------------
MODEL_TURBO_16K = "gpt-3.5-turbo-16k"
MODEL_TURBO_0613 = "gpt-3.5-turbo-0613"
MODEL_GPT4 = "gpt-4-0613"
MODEL_GPT4_1106 = "gpt-4-1106-preview"
# ----------------------------------

# размер чанка, оптимально для Опыта работы - 1024, Обо мне - 2048
chunk_size_we = 1024
chunk_size_am = 2048

clear_output()

class WorkerОpenAI():
  def __init__(self, \
               system_promt = " ", \
               system_promt_assistant = " ", \
               mod = MODEL_TURBO_16K, \
               content_topics = None, \
               save_project = '/content/'):
    self.model = mod
    self.save_project  = save_project

    if content_topics:
      self.content_topics = self.load_txt_file(content_topics)

    # системные настройки
    self.system_promt = self.load_document_text(system_promt)
    self.speaker_system_promt = self.load_document_text(system_promt_assistant)


  def load_document_text(self, url: str) -> str:
      # функция для загрузки документа по ссылке из гугл док
      match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
      if match_ is None:
          raise ValueError('Invalid Google Docs URL')
      doc_id = match_.group(1)
      response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
      response.raise_for_status()
      text = response.text
      return text


  def load_txt_file(self, file_path):
      with open(file_path, 'r') as file_:
          text = file_.read()
      return text

  # пример подсчета токенов
  def num_tokens_from_messages(self, messages):
      """Returns the number of tokens used by a list of messages."""
      try:
          encoding = tiktoken.encoding_for_model(self.model)
      except KeyError:
          encoding = tiktoken.get_encoding("cl100k_base")
      # if self.model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
      if self.model in ["gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-4-0613"]:  # note: future models may deviate from this
          num_tokens = 0
          for message in messages:
              num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
              for key, value in message.items():
                  num_tokens += len(encoding.encode(value))
                  if key == "name":  # if there's a name, the role is omitted
                      num_tokens += -1  # role is always required and always 1 token
          num_tokens += 2  # every reply is primed with <im_start>assistant
          return num_tokens
      else:
          raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {self.model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")


# Для пункта "Опыт работы"
  def create_embedding_one_file_we(self, doc_txt_dir="/content/", \
                                   file_ = "Имя файла.txt" , \
                                   faiss_db_dir ="/content/"):

    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    # Для Копирайтера
    self.splitter = RecursiveCharacterTextSplitter(['\n\n', '\n', ' '], chunk_size=chunk_size_we, chunk_overlap=0)
    chunkID = 0
    count_tokens = 0
    self.file_name = file_[:-3]
    print("Загружается файл: ", file_)
    # проходимся по всем данным
    source_chunks = []
    # разбиваем на несколько частей с помощью метода split_text
    with open(os.path.join(doc_txt_dir, file_), "r") as f:
      for chunk in self.splitter.split_text(f.read()):
          chunkID += 1
          source_chunks.append(Document(page_content=chunk, \
                              metadata={'source': file_,
                                        'chunkID': chunkID}))

    # Создание индексов документа и СОХРАНЕНИЕ
    # Если документ не пуст, то создать и сохранить базу индексов эмбеддингов отрезков документа
    if len(source_chunks) > 0:
        self.db = FAISS.from_documents(source_chunks, OpenAIEmbeddings())
        count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
        count_tokens += count_token
        print('Количество токенов в документе :', count_token)
        # print('ЦЕНА запроса:', 0.0004 * (count_token / 1000), ' $')

        self.db.save_local(os.path.join(faiss_db_dir, f"db_initial_we_{self.file_name[:35]}"))

    print('\nЦЕНА запроса создания базы индексов для пункта "Опыт работы":', 0.0004 * (count_tokens / 1000), ' $')

# Для пункта "Обо мне"
  def create_embedding_one_file_am(self, doc_txt_dir="/content/", \
                                   file_ = "Имя файла.txt" , \
                                   faiss_db_dir ="/content/"):

    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    # Для Копирайтера
    self.splitter = RecursiveCharacterTextSplitter(['\n\n', '\n', ' '], chunk_size=chunk_size_am, chunk_overlap=0)
    chunkID = 0
    count_tokens = 0
    self.file_name = file_[:-3]
    print("Загружается файл: ", file_)
    # проходимся по всем данным
    source_chunks = []
    # разбиваем на несколько частей с помощью метода split_text
    with open(os.path.join(doc_txt_dir, file_), "r") as f:
      for chunk in self.splitter.split_text(f.read()):
          chunkID += 1
          source_chunks.append(Document(page_content=chunk, \
                              metadata={'source': file_,
                                        'chunkID': chunkID}))

    # Создание индексов документа и СОХРАНЕНИЕ
    # Если документ не пуст, то создать и сохранить базу индексов эмбеддингов отрезков документа
    if len(source_chunks) > 0:
        self.db = FAISS.from_documents(source_chunks, OpenAIEmbeddings())
        count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
        count_tokens += count_token
        print('Количество токенов в документе :', count_token)
        # print('ЦЕНА запроса:', 0.0004 * (count_token / 1000), ' $')

        self.db.save_local(os.path.join(faiss_db_dir, f"db_initial_am_{self.file_name[:35]}"))

    print('\nЦЕНА запроса создания базы индексов для пункта "Обо мне":', 0.0004 * (count_tokens / 1000), ' $')

clear_output()

In [ ]:
# загрузим файл со всеми инструкциями от HR отдела
import gdown

# Задаем прямую ссылку на файл (ссылку, которая начинается с "https://drive.google.com/uc?...")
# файл 'merged_resume.txt' доступен по ссылке:  https://drive.google.com/file/d/105lVzXUxliTGU_xgOAL4cz_tqVkNfnOb/view?usp=sharing
file_url = 'https://drive.google.com/uc?id=105lVzXUxliTGU_xgOAL4cz_tqVkNfnOb'

# Путь для сохранения файла в корневой папке Google Colab
output_path = '/content/'  # Можете изменить путь, если нужно сохранить файл в другой папке

# Загрузите файл
gdown.download(file_url, output_path)

Downloading...
From: https://drive.google.com/uc?id=105lVzXUxliTGU_xgOAL4cz_tqVkNfnOb
To: /content/merged_resume.txt
100%|██████████| 346k/346k [00:00<00:00, 87.9MB/s]


'/content/merged_resume.txt'

In [ ]:
# @title Создаем общие чанки из всех инструкций от HR отдела
projects_dir = '/content/'

Promt_copywriter = "https://docs.google.com/document/d/1598tY9blVpyjlZ6YjdT3b2F9zDzNQvgZi7vWGYRRFPY/edit?usp=sharing"
Promt_assistant = "https://docs.google.com/document/d/1tf6HgJvQ05SVSSV59zEKtk-3gkY9pkfAsHizWctwJbU/edit?usp=sharing"


# Создаем объект для дообучения chatGPT
curator_we = WorkerОpenAI(system_promt = Promt_copywriter, # системный промт
                          system_promt_assistant = Promt_assistant, # промт помощника
                          save_project = projects_dir)     # путь для сохранения готовых файлов

curator_am = WorkerОpenAI(system_promt = Promt_copywriter, # системный промт
                          system_promt_assistant = Promt_assistant, # промт помощника
                          save_project = projects_dir)     # путь для сохранения готовых файлов

# Формируем базу по файлу txt
# # путь к материалам
doc_txt_dir = projects_dir
file_name = 'merged_resume.txt' # в файле merged_resume.txt последовательно скопированы все инструкции от HR


curator_we.create_embedding_one_file_we(doc_txt_dir = doc_txt_dir,   # путь к материалам
                                        file_ = file_name,            # какой файл берем
                                        faiss_db_dir = projects_dir)     # путь для сохранения исходной базы
curator_am.create_embedding_one_file_am(doc_txt_dir = doc_txt_dir,   # путь к материалам
                                        file_ = file_name,            # какой файл берем
                                        faiss_db_dir = projects_dir)     # путь для сохранения исходной базы

Загружается файл:  merged_resume.txt
Количество токенов в документе : 82552

ЦЕНА запроса создания базы индексов для пункта "Опыт работы": 0.0330208  $
Загружается файл:  merged_resume.txt
Количество токенов в документе : 82591

ЦЕНА запроса создания базы индексов для пункта "Обо мне": 0.0330364  $


In [ ]:
# Смотрим созданные чанки для пункта "Опыт работы" - этот код можно исключить, так как он чисто информативный
print(f"Текст разбит на чанки. Всего: {len(curator_we.db.docstore._dict.values())} шт.\n")
curator_we.db.docstore._dict.values()

Текст разбит на чанки. Всего: 237 шт.



dict_values([Document(page_content='Принципы хорошего резюме\n\nГлобально резюме состоит из трех информационных блоков:\n\n* Блок с компаниями, в которых вы работали.\n* Блок с технической экспертизой: опыт, знания, умения, навыки.\n* Результаты работы или достижения.\n\nБлок с компаниями. Иногда встречаются резюме, в которых ничего нет, кроме упоминания о работе в «крутой» компании, например, в Яндексе.\n\n￼\n\nДве сухие строки опыта двух лет работы ни о чем говорят. Здесь «крутая» компания играет против кандидата. С таким резюме возникает много вопросов: «А что конкретно он делал два года в Яндекс? За что платить 8 000 евро? За скромный стек? А на каком уровне он им владеет? Чем он будет полезен?» Возможно, это гениальный специалист в своей области, но по резюме это не понятно.\n\nНедостаточно написать о работе в крупной корпорации. Это лишь дополнительный бонус — фишка, из-за которой на резюме уделят больше времени. Бренд работодателя не продает, продает грамотно составленное резюме

In [ ]:
# Смотрим созданные чанки для пункта "Обо мне" - этот код можно исключить, так как он чисто информативный
print(f"Текст разбит на чанки. Всего: {len(curator_am.db.docstore._dict.values())} шт.\n")
curator_am.db.docstore._dict.values()

Текст разбит на чанки. Всего: 123 шт.



dict_values([Document(page_content='Принципы хорошего резюме\n\nГлобально резюме состоит из трех информационных блоков:\n\n* Блок с компаниями, в которых вы работали.\n* Блок с технической экспертизой: опыт, знания, умения, навыки.\n* Результаты работы или достижения.\n\nБлок с компаниями. Иногда встречаются резюме, в которых ничего нет, кроме упоминания о работе в «крутой» компании, например, в Яндексе.\n\n￼\n\nДве сухие строки опыта двух лет работы ни о чем говорят. Здесь «крутая» компания играет против кандидата. С таким резюме возникает много вопросов: «А что конкретно он делал два года в Яндекс? За что платить 8 000 евро? За скромный стек? А на каком уровне он им владеет? Чем он будет полезен?» Возможно, это гениальный специалист в своей области, но по резюме это не понятно.\n\nНедостаточно написать о работе в крупной корпорации. Это лишь дополнительный бонус — фишка, из-за которой на резюме уделят больше времени. Бренд работодателя не продает, продает грамотно составленное резюме

In [ ]:
# иницилизация GPT-4 Turbo
# подробно на https://vc.ru/chatgpt/911997-openai-vypustila-gpt-4-turbo-s-bolee-deshevym-api-i-kontekstnym-oknom-na-128-tysyach-tokenov?ref=recommended
from openai import OpenAI
client = OpenAI()

# Главная роль HR - помощника

Промт для пункта резюме "Обо мне"

In [ ]:
chat_manager_system_am = """
Ты предоставляешь услуги по составлению резюме для кандидатов, нацеленных на позиции специалистов или дата-сайентистов в области искусственного интеллекта.
Как профессионал в своей области, ты должен использовать специальные инструкции для соответствующего составления резюме.
Твоя задача - на основе информации от кандидата и предоставленной тебе инструкции, составить содержательное и качественное описание для раздела "Обо мне" на русском языке, не превышая 200 слов.
Если в инструкции отсутствует необходимая информация, не придумывай что-то своё, а ориентируйся строго на то, что указал кандидат.
Не упоминай в своих ответах о предоставленной тебе инструкции, и используй её как основу при составлении резюме.
Четко следуй поступившим запросам, и не добавляй информацию о качествах специалиста, которые он не указывал в запросе.
Информация от кандидата имеет наивысший приоритет над предоставленной тебе инструкцией, поэтому не бери из инструкции информацию не соответствующую запросу кандидата!
В своей работе фокусируйся на важных деталях резюме. Не включай в ответ информацию о образовании, контактных данных, примечаниях или приветственных оборотах.

Вот формат твоего ответа:
f"Я, {full_name}, ищу позицию {position_sought}, мой опыт работы {experience}. Обо мне {about_me}"
"""

Промт для пункта резюме "Опыт работы"

In [ ]:
chat_manager_system_we = """
Как помощник в составлении резюме, твоя задача, на основе опыта работы кандидата и предоставленной инструкции, помочь ему верно заполнить пункт резюме "Опыт работы".
Создай более осмысленное описание опыта работы, согласно предоставленной тебе инструкции и информации от кандидата, используя русский язык и не превышая 1000 слов.
Не копируй текст из запроса, а раскрой более осмысленно и детально его опыт, но в строгом соответствии с инструкцией и запросом кандидата.
Постоянно ориентируйся на направления, данные кандидатом, и не упоминай в ответах о полученных инструкциях.
Приоритет отдается информации от кандидата, поэтому не используй детали из инструкции, которые противоречат запросу кандидата.
Сконцентрируйся на ключевых деталях резюме и не включай информацию об образовании, контактных данных, замечаниях или приветственных фразах.
Обороты, связанные с участием в хакатонах и митапах или достижения не следует включать, если они не были указаны в запросе кандидата.
Отвечай строго от лица кандидата, учитывая его пол.
Укажи какую позицию и в какой компании кандидат занимал(-а, -ю) на последнем месте работы и при этом правильно применяй окончание глаголов для прошедшего и настоящего времён.
В финальной части укажи, что осуществлял кандидат в процессе своей трудовой деятельности и какие у него были основные обязанности.
В самом конце укажи ключевые навыки кандидата.

Вот формат твоего ответа:
Период: Февраль 2020 - Май 2022
Компания: AppGenius
Должность: Мобильный разработчик
Мои обязанности включали: разработку мобильных приложений для платформ Android и iOS. Я работал в тесном взаимодействии с дизайнерами и другими разработчиками, чтобы создавать инновационные приложения.
В процессе работы в AppGenius я также работал с различными API и интегрировал приложения с внешними сервисами.

Период: Июнь 2022 - Настоящее время
Компания: TechMobile Solutions
Должность: Старший мобильный разработчик
Мои обязанности включают: управление процессом разработки мобильных приложений, распределение задач между разработчиками и обеспечение качества кода.
Я также взаимодействую с клиентами для понимания их требований и помощи в принятии технических решений.

В процессе своей трудовой деятельности я исполнял следующие обязанности: Разработка моделей машинного обучения (Machine Learning Development), Анализ данных и прогнозирование (Data Analysis and Forecasting)

КЛЮЧЕВЫЕ НАВЫКИ
- Разработка мобильных приложений для платформ Android и iOS
- Управление процессом разработки мобильных приложений
- Тесное взаимодействие с дизайнерами и другими разработчиками
- Интеграция приложений с внешними сервисами и работа с различными API
- Распределение задач между разработчиками и обеспечение качества кода
- Взаимодействие с клиентами для понимания их требований
"""

In [ ]:
# @title Функции запроса к chatGPT

def get_chatgpt_answer(topic, point, model = MODEL_GPT4_1106): # MODEL_GPT4, MODEL_TURBO_0613, MODEL_TURBO_16K, MODEL_GPT4_1106
  # выбор нужного промта
  chat_manager_system = """Отвечай строго так на все запросы: Не выбрана главная роль chatGPT!""" # если point не равен 1 или 2 то получим такой ответ от chatGPT
  docs = ""
  if point == 1:
    chat_manager_system = chat_manager_system_am
    # Выборка документов по схожести с вопросом
    docs = curator_am.db.similarity_search(topic, k=6)
  if point == 2:
    chat_manager_system = chat_manager_system_we
    # Выборка документов по схожести с вопросом
    docs = curator_we.db.similarity_search(topic, k=6)
  # print(docs, '\n')
  message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n  ' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
  # print('message_content :\n ======================================== \n', message_content)

  messages = [
    {"role": "system", "content": f"{chat_manager_system}"},
    {"role": "user", "content": f"""Инструкция с информацией для ответа клиенту: {message_content}\n\n Запрос клиента: \n{topic}"""}  # до 12.11.2023 был не "Запрос клиента:", а "Вопрос клиента:"
    ]

  try:
    completion = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.0
    )

    # max_tokens=2000

    # print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    # print('ЦЕНА запроса с ответом :', 0.0015*(completion["usage"]["total_tokens"]/1000), ' $')
    # print('===========================================: \n')
    # print('Ответ ChatGPT: ')
    print(completion.choices[0].message.content)
  except:
    print("Модель в настоящее время перегружена. Попробуйте позже.")

Ниже пойдут функции, для которых нужны данные от сервера (пользователя)

In [ ]:
# например от сервера пришли такие данные от пользователя
full_name = "Иванов Сергей Александрович"
position_sought = "AI Management Consultant"
experience = """2020-2021 ОАО ЛУК, инженер КИП: Сервисное обслуживание приборов КИПиА и систем АСУ ТП.
                2021-2022 ООО ИМС, AI Planning Analyst: Анализ данных, прогнозирование цен акций, разработка статистических моделей.
                2022-по настоящее время ПАО ОЗНА, AI Management Consultant: Разработка сложных моделей машинного обучения для анализа неструктурированных данных."""
about_me = """Программировал на языках С++ и Jawa, разрабатывал c их помощью простые приложения для смартфонов, потом обучался 2 года на программирование нейросетей ML, DL.
              Прошел стажировки в реальных коммерческих проектах по распознованию вредителей на полях с сельхозпродукцией и прогнозировании загрузки серверов в дата-центрах ПАО "Сбербанк".
              Владею такими навыками как знание С++, Jawa, Python, TensorFlow, PyTorch, Keras, AutoML, YOLO, ResNet, Git, MySQL. Опыт работы в команде разработчиков и на удаленке."""
last_job = "ПАО ОЗНА"
last_position = "AI Management Consultant"
tasks_at_previous_jobs = """Сервисное обслуживание приборов КИПиА и систем АСУ ТП. Анализ данных, прогнозирование цен акций, разработка статистических моделей.
                            Разработка сложных моделей машинного обучения для анализа неструктурированных данных."""
key_skills = """Data Analysis and Forecasting, Machine Learning Development, Programming (C++, Java, Python), Proficiency in TensorFlow, PyTorch, and Keras, Experience with AutoML, YOLO, and ResNet
                Team Collaboration and Remote Work, Git and MySQL usage, Statistical Modelling, Maintenance of Control and Measurement Instruments and Automated Control Systems, Unstructured Data Analysis."""


# создадим словарь этих данных
data_gl = {
       "full_name": full_name,
       "position_sought": position_sought,
       "experience": experience,
       "about_me": about_me,
       "last_job": last_job,
       "last_position": last_position,
       "tasks_at_previous_jobs": tasks_at_previous_jobs,
       "key_skills": key_skills
      }

In [ ]:
# функция "обо мне"
def about_me_GPT(data):
    full_name = data["full_name"]
    position_sought = data["position_sought"]
    experience = data["experience"]
    about_me = data["about_me"]
    # соединим все данные "обо мне"
    about_me_text = f"Я, {full_name}, ищу позицию {position_sought}, мой опыт работы {experience}. {about_me}"
    # print(about_me_text)
    # получим ответ от chatGPT после обработки всех данных
    return get_chatgpt_answer(about_me_text, 1)

In [ ]:
# функция "опыт работы"
def experience_GPT(data):
    experience = data["experience"]
    last_job = data["last_job"]
    last_position = data["last_position"]
    tasks_at_previous_jobs = data["tasks_at_previous_jobs"]
    key_skills = data["key_skills"]
    # соединим все данные "опыт работы"
    experience_text = f"""Мой опыт работы {experience}.
                          На последнем месте работы я занимал(-а, -ю) позицию {last_position} в компании {last_job}.
                          В процессе своей трудовой деятельности я исполнял(-а, -ю) следующие обязанности: {tasks_at_previous_jobs}.
                          Мои ключевые навыки включают {key_skills}."""
    # print(experience_text)
    # получим ответ от chatGPT после обработки всех данных
    return get_chatgpt_answer(experience_text, 2)

In [ ]:
# формируем "обо мне"
about_me_GPT(data_gl)

Я, Иванов Сергей Александрович, ищу позицию AI Management Consultant, мой опыт работы включает позиции инженера КИП в ОАО ЛУК с сервисным обслуживанием приборов КИПиА и систем АСУ ТП, AI Planning Analyst в ООО ИМС с анализом данных и прогнозированием цен акций, а также разработкой статистических моделей. С 2022 года работаю AI Management Consultant в ПАО ОЗНА, где занимаюсь разработкой сложных моделей машинного обучения для анализа неструктурированных данных. Имею опыт программирования на С++ и Java, разработки приложений для смартфонов, а также двухлетнее обучение программированию нейросетей ML и DL. Проходил стажировки в проектах по распознаванию вредителей на полях и прогнозированию загрузки серверов в дата-центрах ПАО "Сбербанк". Владею навыками работы с C++, Java, Python, TensorFlow, PyTorch, Keras, AutoML, YOLO, ResNet, Git, MySQL и опытом работы в команде разработчиков и удаленно.


In [ ]:
# формируем "опыт работы"
experience_GPT(data_gl)

ОПЫТ РАБОТЫ

Период: 2020 - 2021
Компания: ОАО "ЛУК"
Должность: Инженер КИП
Мои обязанности включали сервисное обслуживание приборов контроля, измерительных приборов и автоматики (КИПиА), а также систем автоматизированного управления технологическими процессами (АСУ ТП).

Период: 2021 - 2022
Компания: ООО "ИМС"
Должность: AI Planning Analyst
Мои обязанности включали анализ данных, прогнозирование цен акций и разработку статистических моделей для планирования и оптимизации бизнес-процессов.

Период: 2022 - Настоящее время
Компания: ПАО "ОЗНА"
Должность: AI Management Consultant
Мои обязанности включают разработку сложных моделей машинного обучения для анализа неструктурированных данных, что позволяет улучшать процессы принятия решений в компании.

В процессе своей трудовой деятельности я осуществлял следующие обязанности: обслуживание и ремонт приборов КИПиА и систем АСУ ТП, анализ больших объемов данных, прогнозирование экономических показателей, разработка и внедрение статистических м

*************************************************************************************************

Примеры свободных запросов к chatGPT
******************************************************************************************************
topic = """
Программировал на языках С++ и Jawa, разрабатывал c их помощью простые приложения для смартфонов, потом обучался 2 года на программирование нейросетей ML, DL. Прошел стажировки в реальных коммерческих проектах по распознованию
вредителей на полях с сельхозпродукцией и прогнозировании загрузки серверов в дата-центрах ПАО "Сбербанк". Владею такими навыками как знание С++, Jawa, Python, TensorFlow, PyTorch, Keras, AutoML, YOLO, ResNet, Git, MySQL.
Опыт работы в команде разработчиков и на удаленке.
"""

get_chatgpt_answer(topic, 1) # 1 - это раздел "Обо мне"


*************************************************************************************************



topic = """
2020-2021 ОАО ЛУК, инженер КИП: Сервисное обслуживание приборов КИПиА и систем АСУ ТП.
2021-2022 ООО ИМС, AI Planning Analyst: Анализ данных, прогнозирование цен акций, разработка статистических моделей.
2022-по настоящее время ПАО ОЗНА, AI Management Consultant: Разработка сложных моделей машинного обучения для анализа неструктурированных данных.
"""

get_chatgpt_answer(topic, 2) # 2 - это раздел "Опыт работы"